In [1]:
# Select which GPU to use
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.getcwd()

'/mnt/ka_gh_storage/2025_11_27_40x_persister_1HourAmpicillinTreatments'

In [2]:
%load_ext autoreload
%autoreload 2
from segmentation_imports_and_functions import *

/home/ka537/miniconda3/envs/omnipose/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# =============================================================================
# Parameters
# =============================================================================

INPUT_ZARR_PATH  = "trenches.zarr"
OUTPUT_ZARR_PATH = "masks_upscaled.zarr"
LOCK_PATH        = OUTPUT_ZARR_PATH + ".lock" # synchronizer to prevent parallel writing issues, doesn't slow things down much
CONFIG_PATH      = "config.json"

CHANNEL_TO_USE   = "PC" # channel name to use for segmentation
UPSCALE = True          # True for 2x2 upscaling images for segmentation (often significantly improves masks), False to not upscale prior to segmentation
                        # Upscaling more than 2x2 seems to provide negligible benefit, but much larger files, hence we only upscale 2x2

MODEL_NAME = "/home/ka537/external_SSD/Real Training Data/13_09_25 (40x, 6 frames concat, 2x2 upscaled)/models/cellpose_residual_on_style_on_concatenation_off_omni_abstract_nclasses_2_nchan_1_dim_2_13_09_25 (40x, 6 frames concat, 2x2 upscaled)_2025_09_24_00_21_48.111785_epoch_3999"

N_JOBS = 10                   # more isn't significantly faster (be careful not to exceed GPU memory)
OUTPUT_COMPRESSOR = None      # compression slows down significantly, compress later if needed
SILENCE_WORKER_PRINTS = True  # if running on Jupyter, False might crash the notebook :)

# Omnipose parameters
EVAL_KWARGS = dict(
    channels=None,          # channels=None for single-channel (PC) input
    rescale=None,           # set if you need to rescale before segmentation
    mask_threshold=-1,      
    transparency=True,      # transparency in flow output
    flow_threshold=0,       # default is .4; minimal benefit, slower, post-process mask artefacts instead
    omni=True,              # Omnipose mask reconstruction; keep True
    resample=True,          # run dynamics on rescaled grid vs original grid
    verbose=0,              # increase for more outputs
)

# =============================================================================
# Run
# =============================================================================
result = run_segmentation(
    input_zarr_path=INPUT_ZARR_PATH,
    output_zarr_path=OUTPUT_ZARR_PATH,
    lock_path=LOCK_PATH,
    config_path=CONFIG_PATH,
    channel_to_use=CHANNEL_TO_USE,
    model_name=MODEL_NAME,
    n_jobs=N_JOBS,
    output_compressor=OUTPUT_COMPRESSOR,
    eval_kwargs=EVAL_KWARGS,
    upscale=UPSCALE,
    silence_worker_prints=SILENCE_WORKER_PRINTS,
)

print("\nDone.")
print("Wrote:", OUTPUT_ZARR_PATH)
print(f"Processed frames: {result.frames_done:,} in {result.seconds/3600:.2f} h")

2025-12-31 16:29:07,712 [INFO] ** TORCH GPU version installed and working. **
>>> GPU activated? True
Input: trenches_uncompressed.zarr
 shape: (60420, 321, 1, 135, 24) dtype: uint16 chunks: (76, 321, 1, 135, 24)
 upscale: True | output (H,W): (270, 48)
 trench_block (from input chunks[0]): 76
Output: masks_upscaled.zarr
 output shape: (60420, 321, 1, 270, 48)
 output chunks: (76, 321, 1, 270, 48)


chunk blocks:   2%|▍                        | 12/795 [07:15<7:25:30, 34.14s/it]/home/ka537/miniconda3/envs/omnipose/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
chunk blocks:  24%|█████▏                | 188/795 [1:09:38<3:00:53, 17.88s/it]

In [5]:
#